In [189]:
import pandas as pd
import numpy as np
from scipy.linalg import svd
from scipy.sparse import csc_matrix,coo_matrix
import os.path

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [203]:
def usvd(filename):
    if os.path.isfile(filename):
        df = pd.read_csv(filename, sep='\t', header=0, index_col=0) # read the gene X position count-matrix and transpose to have rows<cols
        data = df.values
        '''rescale observed data to (-1,1) range and output the new matrix z'''
        i = np.nonzero(data)[0]
        j = np.nonzero(data)[1]
        x = data[np.nonzero(data)]
        a = x.min()
        b = x.max()
        x = (x - (a+b)*0.5)/((b-a)*0.5)
        z = csc_matrix((x,(i,j)), data.shape).todense()
        '''svd'''
        u, s, vt = svd(z,check_finite=True,full_matrices=False)
        '''truncate svd'''
        p = np.count_nonzero(z)*1.0/z.size # probability estimated from data pattern
        threshold = 2.02*np.sqrt(p*max(z.shape)) # threshold obtained by usvd paper
        ix = np.where( s > threshold )
        k = ix[0].max()+1
        w = np.dot(u[:,:k], np.dot(np.diag(s[:k]), vt[:k,:]))
        '''clean'''
        w[w < -1] = -1
        w[w > 1] = 1
        w = w*(b-a)*0.5+(a+b)*0.5
    else:
        print "Data file not found"
    return w, k

In [204]:
section = 'P4.3'
filename = datadir+section+'.tsv'
usvd(filename)

(array([[42.5083681 , 48.96046961, 47.41875174, ..., 52.00493579,
         50.68737322, 51.24272873],
        [22.55355803, 37.02694064, 33.55586281, ..., 47.82635966,
         41.43523429, 43.24008941],
        [13.11010923, 15.01144521, 14.49573258, ..., 35.11576794,
         18.10623084, 20.97471347],
        ...,
        [52.14038148, 52.24653367, 52.22083993, ..., 52.39951198,
         52.2887975 , 52.31242373],
        [51.84312599, 52.60654191, 52.42501   , ..., 52.68990452,
         52.77359587, 52.80031728],
        [52.31484396, 52.28314414, 52.29032417, ..., 52.39163643,
         52.29127998, 52.30593618]]), 2)

In [206]:
df.values

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 1,  0,  1, ...,  0,  1,  0],
       [24, 13, 16, ...,  3, 10,  4],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])